# Modeling & Fitting

In [1]:
import os
import sys
sys.path.insert(0, "../../")

import mpld3
mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

from core import util, config, database
from core.data.model import ModelDataRepo
from core.data.model.entities import *

database.init()
config.init_plot()

### Get Data

In [2]:
selected_kabko = "KOTA BLITAR"
kabko = ModelDataRepo.get_kabko_full(selected_kabko)

In [3]:
kabko._kapasitas_rs

[(0, 10), (70, 11)]

### Make Model

In [4]:
from core.modeling import SeicrdRlcModel

mod = SeicrdRlcModel(kabko)

### Select Datsets to Use for Fitting

In [5]:
used_datasets = [
    #"infectious", 
    "critical_cared", 
    "infectious_all", 
    "recovered", 
    "dead",
    #"infected"
]
mod.use_datasets(used_datasets)

### List Variables that Can be Fitted

In [6]:
varying_init = [
    "death_rate_over",
    "exposed_rate_over", 
    "kapasitas_rs_mul", 
    "test_coverage_max", 
    "test_coverage_0", "test_coverage_increase", 
    "infectious_rate", 
    "critical_rate", "critical_chance",
    "death_rate_normal", "death_chance_normal", 
    "k", "r_0", "r_1", "r_2"
]

### Select Variables to Fit

In [7]:
varying_now = [
    #"death_rate_over",
    #"exposed_rate_over", 
    #"kapasitas_rs_mul", 
    #"test_coverage_max", 
    #"test_coverage_0", "test_coverage_increase", 
    "infectious_rate", 
    "critical_rate", "critical_chance",
    "death_rate_normal", "death_chance_normal", 
    "k", "r_0", "r_1", "r_2"
]
unvary = [x for x in varying_init if x not in varying_now]

### Fit!

In [8]:
result = mod.fit(test_splits=[], unvary=unvary)
result

### Fit Results

Params Should Have "standard error" and "relative error" Columns. If they dont:
1. A parameter might not be significant
2. A parameter has hit its min/max limit and can't vary

In [9]:
result.fit_result.params

name,value,standard error,relative error,initial value,min,max,vary
days,128.000000,0.00000000,(0.00%),128,-inf,inf,False
population,142798.000,0.00000000,(0.00%),142798,-inf,inf,False
infectious_rate,0.07142857,0.00321722,(4.50%),0.07142857142857144,0.07142857,0.33333333,True
critical_chance,0.62150181,0.15193997,(24.45%),0.6215018053172853,0.01000000,0.66666667,True
critical_rate,0.08333333,0.00186491,(2.24%),0.08333333333333355,0.08333333,0.14285714,True
recovery_rate_normal,0.11111111,0.00000000,(0.00%),0.1111111111111111,-inf,inf,False
recovery_rate_critical,0.05384615,0.00646678,(12.01%),0.053846153846154016,0.05384615,0.15384615,True
death_chance_normal,0.19535877,0.18201009,(93.17%),0.19535877407153945,0.00500000,0.25000000,True
death_rate_normal,0.10000000,0.08943331,(89.43%),0.10000000002992504,0.10000000,0.20000000,True
death_chance_over,1.00000000,0.00000000,(0.00%),1.0,-inf,inf,False


### Model Scores

In [10]:
import math

In [11]:
if result.test_scorer:
    flat_test_scorer = result.test_scorer.flatten()
    print("Test scores")
    print("Max Error: " + str(flat_test_scorer.max_error()))
    print("MAE: " + str(flat_test_scorer.mean_absolute_error()))
    print("MSE: " + str(flat_test_scorer.mean_sqaured_error()))
    print("RMSE: " + str(flat_test_scorer.root_mean_squared_error()))
    print("RMSLE: " + str(flat_test_scorer.root_mean_squared_log_error()))
    print("Explained Variance: " + str(flat_test_scorer.explained_variance_score()))
    print("R2: " + str(flat_test_scorer.r2_score()))
    print()

In [12]:
flat_fit_scorer = result.fit_scorer.flatten()
print("Last fit scores")
print("Max Error: " + str(flat_fit_scorer.max_error()))
print("MAE: " + str(flat_fit_scorer.mean_absolute_error()))
print("MSE: " + str(flat_fit_scorer.mean_squared_error()))
print("RMSE: " + str(flat_fit_scorer.root_mean_squared_error()))
print("RMSLE: " + str(flat_fit_scorer.root_mean_squared_log_error()))
print("Explained Variance: " + str(flat_fit_scorer.explained_variance_score()))
print("R2: " + str(flat_fit_scorer.r2_score()))
print()

AttributeError: 'list' object has no attribute 'flatten'

### Predict

In [ ]:
values = dict(result.fit_result.values)
values["days"] += 30
values["days"]

In [ ]:
model_result = mod.model(**values)

### Plot!

In [ ]:
from core.modeling import ModelPlotter

plotter = ModelPlotter(model_result)

In [ ]:
length = kabko.data_count + result.outbreak_shift
length

In [ ]:
import matplotlib.pyplot as plt
for d in used_datasets:
    datasets = kabko.get_datasets([d], result.outbreak_shift)
    plotter.plot(
        plotter.plot_main_data, 
        datasets,
        length
    )
    plt.show()

In [ ]:
plotter.plot(plotter.plot_main_lite);

In [ ]:
plotter.plot(plotter.plot_daily_lite);

In [ ]:
plotter.plot(plotter.plot_r0);

In [ ]:
plotter.plot(plotter.plot_mortality_rate);

In [ ]:
plotter.plot(plotter.plot_over);

In [ ]:
plotter.plot(plotter.plot_dead);

In [ ]:
plotter.plot(plotter.plot_healthcare);

### Save Params

Run code below ONLY if you want to save the parameters.

In [ ]:
ModelDataRepo.update_all(result.kabko, result.fit_result.params, result.outbreak_shift)

### Confidence Interval

This requires standard error to be present for each params.

result.fit_result.conf_interval()